### 1 - Import and clean test data

In [146]:
import pandas as pd
import pandas_ta as ta
from polygon import RESTClient
import plotly.graph_objects as go
from plotly.offline import plot

client = RESTClient(api_key="n__Yap_rQ8p5ZSPCBnjxBv7zwGymmKHZ")

# Define the ticker symbol and date range

dataRequest = client.get_aggs(ticker = "C:EURUSD",
                              multiplier = 5,
                              timespan = "minute",
                              from_ = "2025-01-01",
                              to = "2100-01-01",
                              limit=50000)

df = pd.DataFrame(dataRequest)

# Convert timestamp to datetime, set as index, and name it "Date"
df = df.set_index(pd.to_datetime(df['timestamp'], unit='ms')).rename_axis("Date")

# Drop the timestamp column as it's now redundant
df = df.drop('timestamp', axis=1)


#df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
#df.set_index(df['Date'], inplace=True)

# Rename columns to have capitalized first letters
df = df.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close'
})

df

#generate plotly
#fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     #open=df['Open'],
                                     #high=df['High'],
                                     #low=df['Low'],
                                     #close=df['Close'])])

#plot(fig, auto_open=True)





,Open,High,Low,Close,volume,vwap,transactions,otc
Date,,,,,,,,
2025-01-01 21:00:00,1.03526,1.03580,1.03526,1.03526,12,1.0354,12,None
2025-01-01 21:05:00,1.03560,1.03580,1.03526,1.03570,17,1.0356,17,None
2025-01-01 21:10:00,1.03526,1.03580,1.03526,1.03526,26,1.0354,26,None
2025-01-01 21:15:00,1.03580,1.03590,1.03526,1.03540,25,1.0356,25,None
2025-01-01 21:20:00,1.03526,1.03550,1.03526,1.03550,4,1.0354,4,None
...,...,...,...,...,...,...,...,...
2025-02-20 16:50:00,1.04760,1.04778,1.04730,1.04756,713,1.0476,713,None
2025-02-20 16:55:00,1.04757,1.04775,1.04730,1.04748,768,1.0476,768,None
2025-02-20 17:00:00,1.04748,1.04760,1.04700,1.04725,925,1.0473,925,None


In [147]:
def populate_indicators(df):
    df["EMA_slow"]=ta.ema(df.Close, length=50)
    df["EMA_fast"]=ta.ema(df.Close, length=30)
    df['RSI']=ta.rsi(df.Close, length=10)
    df['ATR']=ta.atr(df.High, df.Low, df.Close, length=7)
    
    my_bbands = ta.bbands(df.Close, length=15, std=1.5)
    df=df.join(my_bbands)

    return df

df = populate_indicators(df)
df

,Open,High,Low,Close,volume,vwap,transactions,otc,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5
Date,,,,,,,,,,,,,,,,,
2025-01-01 21:00:00,1.03526,1.03580,1.03526,1.03526,12,1.0354,12,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-01 21:05:00,1.03560,1.03580,1.03526,1.03570,17,1.0356,17,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-01 21:10:00,1.03526,1.03580,1.03526,1.03526,26,1.0354,26,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-01 21:15:00,1.03580,1.03590,1.03526,1.03540,25,1.0356,25,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-01 21:20:00,1.03526,1.03550,1.03526,1.03550,4,1.0354,4,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-20 16:50:00,1.04760,1.04778,1.04730,1.04756,713,1.0476,713,None,1.045878,1.046427,66.696054,0.000590,1.046502,1.047065,1.047627,0.107418,0.940401
2025-02-20 16:55:00,1.04757,1.04775,1.04730,1.04748,768,1.0476,768,None,1.045941,1.046495,64.395912,0.000570,1.046543,1.047112,1.047681,0.108731,0.823222
2025-02-20 17:00:00,1.04748,1.04760,1.04700,1.04725,925,1.0473,925,None,1.045992,1.046544,58.005629,0.000575,1.046647,1.047162,1.047677,0.098341,0.585454


In [148]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0

df=df[-10000:-1]
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

100%|██████████| 9999/9999 [00:14<00:00, 698.31it/s]
/tmp/ipykernel_2823533/721800138.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [149]:
df

,Date,Open,High,Low,Close,volume,vwap,transactions,otc,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal
0,2025-01-01 22:25:00,1.03540,1.03570,1.03471,1.03509,34,1.0353,34,None,NaN,NaN,43.712716,0.001000,1.035131,1.035511,1.035892,0.073542,-0.053264,1
1,2025-01-01 22:30:00,1.03570,1.03580,1.03490,1.03560,28,1.0356,28,None,NaN,NaN,56.312286,0.000986,1.035145,1.035525,1.035904,0.073267,0.599293,0
2,2025-01-01 22:35:00,1.03560,1.03580,1.03493,1.03568,108,1.0355,108,None,NaN,NaN,57.952727,0.000969,1.035153,1.035537,1.035920,0.074077,0.686851,0
3,2025-01-01 22:40:00,1.03549,1.03580,1.03540,1.03552,332,1.0355,332,None,NaN,NaN,53.489421,0.000888,1.035155,1.035538,1.035921,0.074036,0.476522,0
4,2025-01-01 22:45:00,1.03550,1.03580,1.03543,1.03560,293,1.0355,293,None,NaN,NaN,55.397809,0.000814,1.035162,1.035518,1.035874,0.068820,0.615064,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,2025-02-20 16:45:00,1.04767,1.04784,1.04736,1.04750,737,1.0477,737,None,1.045810,1.046349,65.873253,0.000609,1.046471,1.047009,1.047547,0.102755,0.956073,2
9995,2025-02-20 16:50:00,1.04760,1.04778,1.04730,1.04756,713,1.0476,713,None,1.045878,1.046427,66.696054,0.000590,1.046502,1.047065,1.047627,0.107418,0.940401,2
9996,2025-02-20 16:55:00,1.04757,1.04775,1.04730,1.04748,768,1.0476,768,None,1.045941,1.046495,64.395912,0.000570,1.046543,1.047112,1.047681,0.108731,0.823222,2
9997,2025-02-20 17:00:00,1.04748,1.04760,1.04700,1.04725,925,1.0473,925,None,1.045992,1.046544,58.005629,0.000575,1.046647,1.047162,1.047677,0.098341,0.585454,2


In [150]:
def total_signal(df, current_candle, backcandles):
    #Uptrend
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    #Downtrend
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 9999/9999 [00:31<00:00, 320.15it/s]
/tmp/ipykernel_2823533/1208225980.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [151]:
df[df.TotalSignal != 0].head(30)

,Date,Open,High,Low,Close,volume,vwap,transactions,otc,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal,TotalSignal
40,2025-01-02 04:30:00,1.03711,1.03715,1.03670,1.03686,362,1.0370,362,None,1.036244,1.036744,45.541168,0.000365,1.036944,1.037187,1.037431,0.046895,-0.172984,2,2
41,2025-01-02 04:35:00,1.03686,1.03690,1.03670,1.03670,240,1.0368,240,None,1.036262,1.036741,40.705729,0.000342,1.036855,1.037159,1.037463,0.058625,-0.254345,2,2
42,2025-01-02 04:40:00,1.03671,1.03675,1.03650,1.03669,284,1.0367,284,None,1.036279,1.036737,40.407785,0.000329,1.036787,1.037136,1.037485,0.067282,-0.139144,2,2
47,2025-01-02 05:05:00,1.03691,1.03692,1.03650,1.03657,334,1.0367,334,None,1.036380,1.036765,36.764001,0.000324,1.036627,1.037036,1.037445,0.078941,-0.069235,2,2
52,2025-01-02 05:30:00,1.03646,1.03660,1.03610,1.03620,257,1.0363,257,None,1.036399,1.036684,28.560653,0.000343,1.036364,1.036753,1.037142,0.075060,-0.210197,2,2
53,2025-01-02 05:35:00,1.03624,1.03639,1.03600,1.03626,198,1.0363,198,None,1.036394,1.036656,31.952920,0.000349,1.036311,1.036687,1.037062,0.072417,-0.068333,2,2
54,2025-01-02 05:40:00,1.03628,1.03640,1.03600,1.03619,272,1.0363,272,None,1.036386,1.036626,30.100134,0.000357,1.036248,1.036625,1.037003,0.072845,-0.076500,2,2
81,2025-01-02 07:55:00,1.03701,1.03702,1.03610,1.03623,754,1.0367,754,None,1.036493,1.036595,38.742422,0.000496,1.036275,1.036665,1.037056,0.075344,-0.057358,2,2
83,2025-01-02 08:05:00,1.03676,1.03690,1.03580,1.03597,1006,1.0365,1006,None,1.036483,1.036565,38.263281,0.000643,1.036219,1.036665,1.037112,0.086132,-0.278739,2,2
84,2025-01-02 08:10:00,1.03596,1.03619,1.03568,1.03580,993,1.0359,993,None,1.036456,1.036515,35.918909,0.000624,1.036081,1.036626,1.037171,0.105106,-0.258106,2,2


In [152]:
import numpy as np
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

/tmp/ipykernel_2823533/2345118794.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [153]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [154]:
def SIGNAL():
    return df.TotalSignal

In [155]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 3000
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=250, margin=1/30)

/tmp/ipykernel_2823533/2590162779.py:36: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [156]:
bt.run()

Start                                     0.0
End                                    9998.0
Duration                               9998.0
Exposure Time [%]                     38.9639
Equity Final [$]                    244.92066
Equity Peak [$]                     295.62981
Return [%]                           -2.03174
Buy & Hold Return [%]                 1.22984
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -29.21997
Avg. Drawdown [%]                    -1.94854
Max. Drawdown Duration                 7468.0
Avg. Drawdown Duration              279.91176
# Trades                                496.0
Win Rate [%]                         39.31452
Best Trade [%]                        0.29859
Worst Trade [%]                      -0.17227
Avg. Trade [%]                    

In [157]:
bt.plot()

GridPlot(id='p1456', ...)